In [1]:
import numpy as np
import pandas as pd
import ta
import ta
import yfinance as yf
import time

In [2]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_FAST_PERIOD = 12
MACD_SLOW_PERIOD = 26
MACD_SIGNAL_PERIOD = 9

In [3]:
def calculate_indicators_macd(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['Close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)
    
    return df

In [4]:
def trading_strategy_test_macd(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Previous_MACD'] < df['Previous_Signal_Line']) & (df['MACD'] >= df['Signal_Line']) & (df['RSI'] > RSI_OVERSOLD)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['Close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['Close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [5]:
start_date='2014-01-01'
end_date='2016-01-01'

In [6]:
non_ESG = ["GME", "FOSL", "GT", "HOG", "PDCO"]

In [13]:
high_ESG =  ['CCU','CNH','TIETO.HE','BAYN.BE','SGSNZ.XC']

In [14]:
medium_esg = ['VOC','VRSK','19MA.F','QNBK.QA','1605.TW']

In [15]:
all_comp = high_ESG + medium_esg + non_ESG

In [10]:
def company_winrate(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)
    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_macd(df)
    winrate = trading_strategy_test_macd(df)
        
    return winrate

In [11]:
winrate_macd = {}
for company in all_comp:
    time.sleep(20)
    try:
        company_winr = company_winrate(company, start_date=start_date, end_date=end_date)
        winrate_macd[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\4122612290.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\4122612290.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [16]:
company_winr = company_winrate('CNH', start_date=start_date, end_date=end_date)
winrate_macd['CNH'] = company_winr

C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\4122612290.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\4122612290.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [18]:
winrate_macd.pop('CNHI.VI')

[0, 0]

In [19]:
winrate_macd

{'CCU': {5: 50.0,
  10: 47.82608695652174,
  15: 43.47826086956522,
  20: 65.21739130434783,
  25: 47.82608695652174,
  30: 43.47826086956522},
 'TIETO.HE': {5: 65.21739130434783,
  10: 63.63636363636363,
  15: 59.09090909090909,
  20: 63.63636363636363,
  25: 68.18181818181817,
  30: 71.42857142857143},
 'BAYN.BE': {5: 70.58823529411765,
  10: 58.82352941176471,
  15: 52.94117647058824,
  20: 52.94117647058824,
  25: 58.82352941176471,
  30: 62.5},
 'SGSNZ.XC': {5: 41.17647058823529,
  10: 47.05882352941176,
  15: 41.17647058823529,
  20: 52.94117647058824,
  25: 43.75,
  30: 37.5},
 'VOC': {5: 50.0, 10: 50.0, 15: 30.0, 20: 30.0, 25: 30.0, 30: 50.0},
 'VRSK': {5: 47.05882352941176,
  10: 64.70588235294117,
  15: 70.58823529411765,
  20: 47.05882352941176,
  25: 70.58823529411765,
  30: 70.58823529411765},
 '19MA.F': {5: 61.53846153846154,
  10: 53.84615384615385,
  15: 46.15384615384615,
  20: 53.84615384615385,
  25: 61.53846153846154,
  30: 69.23076923076923},
 'QNBK.QA': {5: 37.5,


In [20]:
winrate_macd = pd.DataFrame(winrate_macd)

In [22]:
winrate_macd.to_excel('winrate_result/macd.xlsx')

In [23]:
def calculate_indicators_bb(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['Close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['Close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    return df

In [24]:
def trading_strategy_test_bb(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Close'] <= df['Bollinger_low']) & (df['RSI'] < RSI_OVERSOLD)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['Close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['Close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [25]:
def company_winrate_bb(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)

    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_bb(df)
    winrate = trading_strategy_test_bb(df)
        
    return winrate

In [26]:
winrate_bb = {}
for company in all_comp:
    time.sleep(12)
    try:
        company_winr = company_winrate_bb(company, start_date=start_date, end_date=end_date)
        winrate_bb[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\402886830.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\402886830.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [27]:
winrate_bb

{'CCU': {5: 71.42857142857143,
  10: 71.42857142857143,
  15: 66.66666666666666,
  20: 66.66666666666666,
  25: 83.33333333333334,
  30: 100.0},
 'CNH': {5: 41.66666666666667,
  10: 41.66666666666667,
  15: 41.66666666666667,
  20: 25.0,
  25: 25.0,
  30: 33.33333333333333},
 'TIETO.HE': {5: 80.0, 10: 60.0, 15: 80.0, 20: 80.0, 25: 80.0, 30: 100.0},
 'BAYN.BE': {5: 100.0,
  10: 100.0,
  15: 100.0,
  20: 66.66666666666666,
  25: 66.66666666666666,
  30: 66.66666666666666},
 'SGSNZ.XC': {5: 63.63636363636363,
  10: 63.63636363636363,
  15: 81.81818181818183,
  20: 81.81818181818183,
  25: 72.72727272727273,
  30: 72.72727272727273},
 'VOC': {5: 23.52941176470588,
  10: 17.647058823529413,
  15: 17.647058823529413,
  20: 26.666666666666668,
  25: 20.0,
  30: 20.0},
 'VRSK': {5: 50.0, 10: 75.0, 15: 75.0, 20: 75.0, 25: 100.0, 30: 100.0},
 '19MA.F': {5: 50.0,
  10: 58.333333333333336,
  15: 58.333333333333336,
  20: 58.333333333333336,
  25: 75.0,
  30: 50.0},
 'QNBK.QA': {5: 75.0,
  10: 66.6

In [28]:
winrate_bb = pd.DataFrame(winrate_bb)

In [29]:
winrate_bb.to_excel('winrate_result/bb.xlsx')

In [30]:
OBV_PERIOD = 5

In [31]:
def calculate_indicators_obv(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [32]:
def trading_strategy_test_obv(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['Close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['Close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [33]:
def company_winrate_obv(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)

    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_obv(df)
    winrate = trading_strategy_test_obv(df)
        
    return winrate

In [34]:
winrate_obv = {}
for company in all_comp:
    time.sleep(12)
    try:
        company_winr = company_winrate_obv(company, start_date=start_date, end_date=end_date)
        winrate_obv[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\1922382557.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9976\1922382557.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [35]:
winrate_obv = pd.DataFrame(winrate_obv)
winrate_obv.to_excel('winrate_result/obv.xlsx')